In [8]:
import pandas as pd

score_files = [
    "AASIST_no_laundering_no_laundering_eval_scores.txt",
    "cqcc-gmm-512-asvspoof19-LA-scores-labels.txt",
    "LFCC_LCNN_eval_ASVspoof2019_LA_eval_score-labels.txt",
    "OCSoftmax_no_laundering_no_laundering_checkpoint_cm_score-labels.txt",
    "RawGAT_ASVspoof2019_LA_eval_eval_CM_scores-labels.txt",
    "RawNet2_ASVspoof2019_LA_eval_eval_CM_scores-labels.txt"
]

initial_weights = {
    "AASIST_no_laundering_no_laundering_eval_scores.txt": 0.4,
    "cqcc-gmm-512-asvspoof19-LA-scores-labels.txt": 0.1,
    "LFCC_LCNN_eval_ASVspoof2019_LA_eval_score-labels.txt": 0.1,
    "OCSoftmax_no_laundering_no_laundering_checkpoint_cm_score-labels.txt": 0.1,
    "RawGAT_ASVspoof2019_LA_eval_eval_CM_scores-labels.txt": 0.2,
    "RawNet2_ASVspoof2019_LA_eval_eval_CM_scores-labels.txt": 0.1
}

def load_scores(file_path):
    scores = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 3:
                identifier, label, score = parts[0], parts[1], float(parts[2])
                scores[identifier] = (label, score)
    return scores

def compute_weighted_average_predictions(score_files, model_weights):
    combined_scores = {}
    for file_path in score_files:
        scores = load_scores(file_path)
        weight = model_weights.get(file_path, 0)
        for identifier, (label, score) in scores.items():
            if identifier not in combined_scores:
                combined_scores[identifier] = {'label': label, 'weighted_scores': 0, 'total_weight': 0}
            combined_scores[identifier]['weighted_scores'] += score * weight
            combined_scores[identifier]['total_weight'] += weight
    
    avg_predictions = {}
    for identifier, data in combined_scores.items():
        avg_score = data['weighted_scores'] / data['total_weight'] if data['total_weight'] > 0 else 0
        label = data['label']
        predicted_label = 'bonafide' if avg_score > 0 else 'spoof'
        avg_predictions[identifier] = {'label': label, 'avg_score': avg_score, 'predicted_label': predicted_label}
    
    return avg_predictions, combined_scores

def update_weights(score_files, avg_predictions_df):
    errors = {}
    for file_path in score_files:
        scores = load_scores(file_path)
        FP, FN, accuracy = 0, 0, 0
        total_samples = len(scores)
        for identifier, (label, score) in scores.items():
            predicted_label = 'bonafide' if score > 0 else 'spoof'
            if predicted_label != label:
                if label == 'bonafide':
                    FP += 1
                else:
                    FN += 1
        accuracy = (total_samples - (FP + FN)) / total_samples * 100
        errors[file_path] = {'FP': FP / total_samples, 'FN': FN / total_samples, 'Accuracy': accuracy / 100}
    
    updated_weights = {}
    file_paths = list(errors.keys())
    accuracies = [errors[fp]['Accuracy'] for fp in file_paths]
    sum_accuracies = sum(accuracies)

    if sum_accuracies > 0:
        for file_path in file_paths:
            updated_weights[file_path] = errors[file_path]['Accuracy'] / sum_accuracies
    else:
        for file_path in file_paths:
            updated_weights[file_path] = 1 / len(file_paths)
    
    return updated_weights

def analyze_accuracy(avg_predictions):
    total = len(avg_predictions)
    correct = sum(1 for data in avg_predictions.values() if data['label'] == data['predicted_label'])
    avg_accuracy = correct / total * 100 if total > 0 else 0

    print(f"Average Prediction Accuracy: {avg_accuracy:.2f}%")    
    return avg_accuracy

# Initial weight setting
model_weights = initial_weights

# Continuous weight update loop
n_iterations = 10  # Number of iterations for updating weights
for iteration in range(n_iterations):
    print(f"\nIteration {iteration + 1} with Weights:")
    for k, v in model_weights.items():
        print(f"{k}: {v:.4f}")

    # Compute weighted avg predictions with current weights
    avg_predictions, combined_scores = compute_weighted_average_predictions(score_files, model_weights)
    avg_accuracy = analyze_accuracy(avg_predictions)

    # Dynamic weight update based on performance
    print("\nUpdating Weights...")
    model_weights = update_weights(score_files, avg_predictions)

print("\nFinal Weights after Iterations:")
for k, v in model_weights.items():
    print(f"{k}: {v:.4f}")

print("\nFinal Results with Updated Weights:")
avg_predictions, combined_scores = compute_weighted_average_predictions(score_files, model_weights)
avg_accuracy = analyze_accuracy(avg_predictions)

# Adding the avg Prediction Accuracy to the comparison DataFrame
avg_accuracy_row = pd.DataFrame({'File': ['Average Prediction'], 'Accuracy': [avg_accuracy]})
comparison_df = pd.DataFrame(avg_accuracy_row)

print(comparison_df)


Iteration 1 with Weights:
AASIST_no_laundering_no_laundering_eval_scores.txt: 0.4000
cqcc-gmm-512-asvspoof19-LA-scores-labels.txt: 0.1000
LFCC_LCNN_eval_ASVspoof2019_LA_eval_score-labels.txt: 0.1000
OCSoftmax_no_laundering_no_laundering_checkpoint_cm_score-labels.txt: 0.1000
RawGAT_ASVspoof2019_LA_eval_eval_CM_scores-labels.txt: 0.2000
RawNet2_ASVspoof2019_LA_eval_eval_CM_scores-labels.txt: 0.1000
Average Prediction Accuracy: 97.98%

Updating Weights...

Iteration 2 with Weights:
AASIST_no_laundering_no_laundering_eval_scores.txt: 0.1875
cqcc-gmm-512-asvspoof19-LA-scores-labels.txt: 0.1097
LFCC_LCNN_eval_ASVspoof2019_LA_eval_score-labels.txt: 0.1719
OCSoftmax_no_laundering_no_laundering_checkpoint_cm_score-labels.txt: 0.1682
RawGAT_ASVspoof2019_LA_eval_eval_CM_scores-labels.txt: 0.1890
RawNet2_ASVspoof2019_LA_eval_eval_CM_scores-labels.txt: 0.1737
Average Prediction Accuracy: 96.82%

Updating Weights...

Iteration 3 with Weights:
AASIST_no_laundering_no_laundering_eval_scores.txt: 0.1